In [10]:
import pandas as pd
import altair as alt
import json

# Read in configuration details
with open("../cache/ny-test-3b-modes/config.json") as infile:
    config = json.load(infile)

# Read in summary data
summary = pd.read_csv("../cache/ny-test-3b-modes/summary.csv")
summary.head()

,metric,pop_total,pop_white,pop_black,pop_asiapacific,hhld_single_mother
0,pop_total_c30_0,3.162418e+05,3.116983e+05,3.178701e+05,3.002934e+05,3.099159e+05
1,pop_total_c45_0,1.213002e+06,1.208124e+06,1.229111e+06,1.060976e+06,1.198415e+06
2,pharmacies_t3_0,9.004288e+00,9.688207e+00,9.147544e+00,7.616010e+00,9.164228e+00
3,hospitals_t1_0,2.272163e+01,2.283817e+01,2.265457e+01,2.377017e+01,2.279459e+01
4,urgent_care_facilities_t1_0,2.701263e+01,2.460297e+01,2.810664e+01,2.739320e+01,2.685053e+01


## Differences in Population Demographics Across A Single Variable

In [11]:
# Let's start by fetching only the data that isn't a comparison
absolutes = summary[summary['metric'].str[-3:] != "1-0"].copy()
# Next, we split our data into scenario columns
absolutes['scenario'] = absolutes["metric"].str[-1]
# to_plot['metric_name'] = 
absolutes['parameter'] = absolutes["metric"].str[:-2].str.split("_").str[-1]
absolutes['opportunity'] = absolutes["metric"].str[:-2].str.split("_").str[:-1]
absolutes['opportunity'] = absolutes['opportunity'].str.join("_")
absolutes = absolutes.drop(columns=['metric'])
absolutes = absolutes.melt(id_vars=["opportunity", "parameter", "scenario"])
opp_dict = {}
for k in config["opportunities"].keys():
    opp_dict[k] = config["opportunities"][k]["name"]

scen_dict = {}
for idx, s in enumerate(config["scenarios"]):
    scen_dict[str(idx)] = s['name']

absolutes["scenario"] = absolutes["scenario"].map(scen_dict)
absolutes['variable'] = absolutes['variable'].map(config["demographics"])
absolutes['opportunity'] = absolutes['opportunity'].map(opp_dict)
absolutes.head()

,opportunity,parameter,scenario,variable,value
0,Total Population,c30,All Modes,Total Population,3.162418e+05
1,Total Population,c45,All Modes,Total Population,1.213002e+06
2,Pharmacies,t3,All Modes,Total Population,9.004288e+00
3,Hospitals,t1,All Modes,Total Population,2.272163e+01
4,Urgent Care Facilities,t1,All Modes,Total Population,2.701263e+01


In [12]:
to_plot = absolutes.query("opportunity == 'Total Population' and parameter == 'c45'").copy()

bars = alt.Chart().mark_bar(color="lightgrey", height=3).encode(
    alt.X("value:Q", title="Total Jobs Accessible in 45 Minutes"),
    alt.Y("scenario:N", axis=alt.Axis(labels=False, title=None))
)

circles = alt.Chart().mark_circle(size=150, opacity=1).encode(
    alt.X("value:Q"),
    alt.Y("scenario:N"),
    alt.Color("scenario:N", title="Scenario")
)


alt.layer(bars, circles, data=to_plot).facet(
    row=alt.Row("variable:N", title=None, header=alt.Header(labelAngle=0, labelAlign="left"))
).properties(
    title={
        "text": config["project"],
        "subtitle": "Measured for 7-9am on Friday, February 3, 2023"
    }
)

alt.FacetChart(...)

In [13]:
# Let's start by fetching only the data that isn't a comparison
deltas = summary[summary['metric'].str[-3:] == "1-0"].copy()

# to_plot['metric_name'] = 
deltas['parameter'] = deltas["metric"].str[:-2].str.split("_").str[-2]
deltas['opportunity'] = deltas["metric"].str[:-2].str.split("_").str[:-2].str.join("_")
# deltas['opportunity'] = deltas['opportunity'].str.join("_")
deltas = deltas.drop(columns=['metric'])
deltas = deltas.melt(id_vars=["opportunity", "parameter"])

# opp_dict = {}
# for k in config["opportunities"].keys():
#     opp_dict[k] = config["opportunities"][k]["name"]

deltas['variable'] = deltas['variable'].map(config["demographics"])
deltas['opportunity'] = deltas['opportunity'].map(opp_dict)
deltas.head()

,opportunity,parameter,variable,value
0,Total Population,c30,Total Population,-112318.765636
1,Total Population,c45,Total Population,-596995.400468
2,Pharmacies,t3,Total Population,0.039859
3,Hospitals,t1,Total Population,2.081188
4,Urgent Care Facilities,t1,Total Population,3.663335


In [14]:
to_plot = deltas.query("opportunity == 'Total Population' and parameter == 'c45'").copy()

bars = alt.Chart(to_plot).mark_bar(color="lightgrey", height=3).encode(
    alt.X("value:Q", title="Change in Jobs Accessible in 45 Minutes"),
    alt.Y("variable:N", title=None)
)

circles = alt.Chart(to_plot).mark_circle(color="#823BA0", size=150, opacity=1).encode(
    alt.X("value:Q"),
    alt.Y("variable:N")
)

(bars+circles).properties(
    title={
        "text": config["project"],
        "subtitle": "Measured for 7-9am on Friday, February 3, 2023"
    },
    height=100, 
    width=600
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    anchor="start"
)

alt.LayerChart(...)

### Small Multiples

In [15]:
to_plot = deltas.copy()
to_plot['combined_name'] = deltas['opportunity'] + " (" + deltas["parameter"] + ")"

bars = alt.Chart().mark_bar(color="lightgrey", height=3).encode(
    alt.X("value:Q", title="Change in Measure"),
    alt.Y("variable:N", title=None)
)

circles = alt.Chart().mark_circle(color="#559613", size=150, opacity=1).encode(
    alt.X("value:Q"),
    alt.Y("variable:N")
)

alt.layer(bars, circles, data=to_plot).facet(
    facet=alt.Facet("combined_name:N", title=None, header=alt.Header(labelFontWeight='bold')),
    columns=2
).properties(
    title={
        "text": config["project"],
        "subtitle": "Measured for 7-9am on Friday, February 3, 2023"
    }
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    fontSize=18,
    subtitleFontSize=14
).resolve_scale(x='independent')

alt.FacetChart(...)

In [18]:
pct_delta = pd.merge(
    absolutes[absolutes['scenario'] == "With Subway"][["opportunity", "parameter", "variable", "value"]], 
    absolutes[absolutes['scenario'] == "Without Subway"][["opportunity", "parameter", "variable", "value"]], 
    on=["opportunity", "parameter", "variable"], suffixes=["0", "1"]
)

pct_delta['pct_delta'] = 100* (pct_delta["value1"] - pct_delta["value0"])/pct_delta["value0"]
pct_delta = pct_delta.drop(columns=["value0", "value1"])
pct_delta.head()
    

,opportunity,parameter,variable,pct_delta
0,Total Population,c30,Total Population,0.0
1,Total Population,c45,Total Population,0.0
2,Pharmacies,t3,Total Population,0.0
3,Hospitals,t1,Total Population,0.0
4,Urgent Care Facilities,t1,Total Population,0.0


In [19]:
to_plot = pct_delta.copy()
to_plot['combined_name'] = deltas['opportunity'] + " (" + deltas["parameter"] + ")"

bars = alt.Chart().mark_bar(color="lightgrey", height=3).encode(
    alt.X("pct_delta:Q", title="% Change in Measure"),
    alt.Y("variable:N", title=None)
)

circles = alt.Chart().mark_circle(color="#2464B0", size=150, opacity=1).encode(
    alt.X("pct_delta:Q"),
    alt.Y("variable:N")
)

alt.layer(bars, circles, data=to_plot).facet(
    facet=alt.Facet("combined_name:N", title=None, header=alt.Header(labelFontWeight='bold')),
    columns=2
).properties(
    title={
        "text": config["project"],
        "subtitle": "Measured for 7-9am on Friday, February 3, 2023"
    }
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    fontSize=18,
    subtitleFontSize=14
)

alt.FacetChart(...)

In [9]:
m0 = pd.read_csv("../cache/ny-test-3b-modes/matrix0.csv")
print(m0.shape[0], m0[m0.travel_time.isna()].shape[0])
m1 = pd.read_csv("../cache/ny-test-3b-modes/matrix0.csv")
print(m1.shape[0], m1[m1.travel_time.isna()].shape[0])

23483716 998292
23483716 998292
